In [11]:
import pandas as pd
import biotda.mapper_pipline as mapper_pipline
import biotda.preprocessing as preprocessing
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import json
import biotda.main as main

## Single-nucleus chromatin accessibility and transcriptomic characterization of Alzheimer’s Disease (GSE174367)

Experiment type:	Expression profiling by high throughput sequencing. Genome binding/occupancy profiling by high throughput sequencing

The gene-regulatory landscape of the brain is highly dynamic in health and disease, coordinating a menagerie of biological processes across¬ distinct cell-types. Here, we present a multi-omic single-nucleus study of 191,890 nuclei in late-stage Alzheimer’s Disease (AD), accessible through our intuitive web portal, profiling chromatin accessibility and gene expression in the same biological samples and uncovering vast cellular heterogeneity. We identified cell-type specific, disease-associated candidate cis-regulatory elements and their candidate target genes, including an oligodendrocyte-associated regulatory module containing links to APOE and CLU. We describe cis-regulatory relationships in specific cell-types at AD risk loci defined by genome wide association studies (GWAS), demonstrating the utility of this multi-omic single-nucleus approach. Trajectory analysis of glial populations identified disease-relevant transcription factors, like SREBF1 and their regulatory targets. Further, we introduce scWGCNA, a co-expression network analysis strategy robust to the sparsity of single-cell data, to perform a systems-level meta-analysis of AD transcriptomics.
 	
Morabito S, Miyoshi E, Michael N, Shahin S et al. Single-nucleus chromatin accessibility and transcriptomic characterization of Alzheimer's disease. Nat Genet 2021 Aug;53(8):1143-1155. PMID: 34239132

### Reading preprocessed data

In [12]:
d_merged=pd.read_csv('data\\metabolomics\\data.csv').set_index('Unnamed: 0').T
d_merged.head()

Unnamed: 0,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000283597,ENSG00000283619,ENSG00000283623,ENSG00000283633,ENSG00000283635,ENSG00000283646,ENSG00000283652,ENSG00000283662,ENSG00000283674,ENSG00000283683
Sample-1,7.316251,4.274548,7.262058,6.663354,5.221161,6.813789,6.806842,9.458696,10.129412,7.288785,...,4.340793,5.641259,3.754220,7.674109,7.034334,5.624611,9.289292,6.550443,7.916829,4.786196
Sample-10,6.808974,5.260135,7.115016,7.203029,4.165419,6.460773,6.812336,9.905395,10.617105,7.933976,...,3.712015,5.428915,4.434448,8.343289,7.447873,5.615707,8.571755,6.627103,7.709677,5.008819
Sample-100,6.552535,4.804724,6.923650,6.245148,5.204919,6.162263,6.554665,9.717658,10.110915,7.609833,...,3.305795,6.091046,4.507657,7.767780,6.598748,5.564879,7.751093,7.101492,8.273221,5.316456
Sample-101,7.073471,5.457774,7.012125,7.053250,4.402102,5.990521,6.686977,9.700598,10.393530,8.135075,...,3.223861,6.341113,4.207010,8.455561,7.629895,5.230453,8.856678,6.931344,8.200737,4.812053
Sample-11,6.758017,4.729599,6.839936,7.046774,4.155380,6.883295,6.468586,9.358712,10.131947,7.556673,...,4.076035,5.730058,5.031616,7.723930,7.334398,5.295999,9.172799,6.864444,8.265620,4.839428


In [13]:
metadata=pd.read_csv('data\\metabolomics\\target.csv').set_index('SampleID')
metadata.head()

,Unnamed: 0,RNAIsolation.Group,Dissection.Group,Sample.ID,Case.Year,Case.Num,Region,Neuropath.Dx.1,Neuropath.Dx.2,Age,...,MEDIAN_5PRIME_TO_3PRIME_BIAS,TOTAL_CLUSTERS,ALIGNED_READS,AT_DROPOUT,GC_DROPOUT,Seq.PC1,Seq.PC2,Seq.PC3,Seq.PC4,Seq.PC5
SampleID,,,,,,,,,,,,,,,,,,,,,
Sample-1,1,Group16,Group7,1,2000,8,FC,Alzheimer's disease,Unused,82,...,0.870195,42669514,65203198,20.803772,0.163871,-0.133179,-0.103026,0.412722,0.423658,-0.529204
Sample-10,2,Group13,Group13,10,2002,4,FC,Alzheimer's disease,Unused,83,...,0.543860,26814831,46094008,3.724324,3.087543,0.027563,0.108425,-0.009570,0.085121,0.056185
Sample-100,3,Group5,Group17,100,1999,26,FC,Normal (Mild Braak Changes),Unused,79,...,0.567146,32645219,59133696,7.030072,1.990039,-0.064691,-0.033957,-0.093489,0.026464,-0.033087
Sample-101,4,Group8,Group6,101,1999,37,FC,Normal (Mild Braak Changes),Unused,74,...,0.737687,24149702,44736556,11.776616,1.562165,0.051882,-0.150986,-0.042881,-0.068372,-0.000339
Sample-11,13,Group2,Group17,11,2002,16,FC,Alzheimer's disease,Unused,90,...,0.455994,26803406,50174354,3.197977,3.365408,0.008923,0.018487,-0.126085,-0.020246,-0.061431


### Setting paramerers for program

In [14]:
#columns we consider as covariates
covariates_columns=['Age', 'Sex']

In [15]:
#factores we want to study
p=d_merged.columns
len(p)

18849

In [16]:
d_merged[covariates_columns]=metadata.loc[d_merged.index, covariates_columns].replace({'M':0, 'F':1}).values

In [17]:
#array with indexes for "normal" samples and "disease" samples
N=metadata.index[metadata['Neuropath.Dx.1']=="Alzheimer's disease"]
D=metadata.index[metadata['Neuropath.Dx.1']!="Alzheimer's disease"]

In [18]:
#check and fix batch effect on batch_column (the column indicates datasets)
fix_batch_effect=False
batch_column=None

#which filter functions to use
filters=[
    'DSGA', 
    'KNearest', 
    'MDS', 
    'PCA', 
    'PCAflat'
]

#where to save results or look for previously culculated data
folder_name='./images/AD/' 

#path to the K-nearest neighbors distance matrix if it have been calculated 
knn_dist_path=None

#cut-off for normal samples to reduce number of samples to speed up calculations 
n_s1=None
#cut-off for disease samples 
n_s2=None


Parsing results if avalable to extract factors significatly differs in subgroupes of samples. This is needed for the persistance homology pipline to reduce dimention. We consider most wariable and significatly different features. You can specify other fetures in this list. If the empty list is given the same same function would be performed. 

In [19]:
sign=main.res_scrap(folder_name)

### Perform calculations

In [ ]:
main.run(d_merged, D, N, list(p), list(covariates_columns), 
         folder_name, fix_batch_effect, batch_column,
        filters,
        knn_dist_path,
        n_s1,
        n_s2, 
        overlap_l=20, #lower edge to check for overlap in the Mapper
        PH=True, #Either to perform the persistance homology pipline
        sign=sign
       )

DSGA
PH pipline
KeplerMapper(verbose=1)
Mapping on data shaped (90, 18851) using lens shaped (90, 2)

Creating 225 hypercubes.

Created 49 edges and 31 nodes in 0:00:00.394675.
Something wrong 0
KNearest
PH pipline
KeplerMapper(verbose=1)
Mapping on data shaped (90, 18851) using lens shaped (90, 2)

Creating 169 hypercubes.

Created 242 edges and 82 nodes in 0:00:00.825070.
